In [2]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193668 sha256=f49969bad4958eab4ee5ab0f0540e117912b0b044b781e32fe58c753bd225af1
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [3]:
from surprise import Dataset, NormalPredictor, Reader,SVDpp,KNNBasic,accuracy,CoClustering
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import random
import pandas as pd

Importing offline data

In [4]:
cols=['NaN','index','timestamp','userID','data_str','movieID','rating'] 
df = pd.read_csv('cleaned_data.csv',names = cols,header=None)

Importing into Readable dataset

In [5]:
df = df.tail(-1)

reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
dataset = Dataset.load_from_df(df[["userID", "movieID", "rating"]], reader)

train, test = train_test_split(dataset, test_size=0.25)

SVD predictions

In [6]:
algo =  SVDpp()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(train)
predictions = algo.test(test)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.7187


0.718713327754466

Extra User Details rendered offline

In [7]:
test_user_details = pd.read_csv('user_details.csv')

Segregating the test data into buckets of different age groups:
1. Below 25
2. Between 25 to 45
3. Above 45

In [33]:
bucket1 = (test_user_details['age']<=25)
bucket2 = ((test_user_details['age']>25) & (test_user_details['age']<=45))
bucket3 = (test_user_details['age']>45)

In [37]:
under_25 = list(test_user_details[bucket1]['user_id'])
from25_to_45 = list(test_user_details[bucket2]['user_id'])
after_45 = list(test_user_details[bucket3]['user_id'])

In [65]:
print(len(under_25),len(from25_to_45),len(after_45))

66616 300024 12754


Randomly sampling 2000

In [50]:
testsample = random.sample(test,2000)

Taking a subsection of each of the buckets of the age

In [57]:
test_under_25 = [row for row in test if int(row[0]) in under_25]

In [58]:
test_25_to_45 = [row for row in test if int(row[0]) in from25_to_45]

In [59]:
test_after45 = [row for row in test if int(row[0]) in after_45]

Predictions for first age bucket

In [60]:
predictions_1 = algo.test(test_under_25)
accuracy.rmse(predictions_1)

RMSE: 0.7473


0.7473418391040447

Second

In [61]:
predictions_2 = algo.test(test_25_to_45)
accuracy.rmse(predictions_2)

RMSE: 0.7134


0.7134302521867458

Third

In [62]:
predictions_3 = algo.test(test_after45)
accuracy.rmse(predictions_3)

RMSE: 0.6657


0.665703982175832

## K-Fold Cross Validation

In [63]:
from surprise.model_selection import cross_validate

cross_validate(algo,dataset,measures=['RMSE'],cv=5, verbose=True, n_jobs=-1)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7002  0.7175  0.7278  0.7410  0.7162  0.7205  0.0135  
Fit time          0.37    0.41    0.83    0.81    0.57    0.60    0.19    
Test time         0.07    0.12    0.06    0.05    0.04    0.07    0.03    


{'test_rmse': array([0.70017728, 0.71751575, 0.72775333, 0.74104199, 0.71618765]),
 'fit_time': (0.3690822124481201,
  0.40718770027160645,
  0.8257157802581787,
  0.8123235702514648,
  0.5745441913604736),
 'test_time': (0.06761336326599121,
  0.12447166442871094,
  0.06334519386291504,
  0.05435371398925781,
  0.03987598419189453)}